In [507]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import requests

from datetime import datetime
import time

from scrapy import Selector
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [200]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_df = pd.DataFrame()

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    company_name = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[1]/a/@title'.format(n)).get()
    industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    sub_industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    new_list = [ticker, company_name, industry, sub_industry]
    pd_series_row = pd.Series(new_list, index = ['ticker', 'company_name', 'industry', 'sub_industry'])
    nasdaq100_df = nasdaq100_df.append(pd_series_row, ignore_index=True)
    print(ticker, end = ' ')

nasdaq100_df.head(4)

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM 

,company_name,industry,sub_industry,ticker
0,Activision Blizzard,Communication Services,Communication Services,ATVI
1,Adobe Inc.,Information Technology,Information Technology,ADBE
2,Advanced Micro Devices,Information Technology,Information Technology,AMD
3,Align Technology,Health Care,Health Care,ALGN


In [438]:
def unix_to_date(unix_timestamp):
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [439]:
with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

In [446]:
def get_income_statement(ticker, period):
    financials_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-financials"

    querystring = {"symbol":ticker,"region":"US"}

    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': KEY
        }

    financials_response = requests.request("GET", financials_url, headers=headers, params=querystring).json()

    ##these are quarterly earnings specifically
    income_statment_df = pd.DataFrame()

    for quarter in financials_response['incomeStatementHistoryQuarterly']['incomeStatementHistory']:
        revenue_line = pd.Series([quarter['endDate']['fmt'],
                                  round(quarter['totalRevenue']['raw']/1000000000,3),
                                  round(quarter['grossProfit']['raw']/1000000000,3),
                                  round(quarter['netIncome']['raw']/1000000000,3),
                                  ],
        index = ['quarter','revenue_billion','gross_profit_billion','net_income_billion'])
        income_statment_df = income_statment_df.append(revenue_line, ignore_index=True)

    income_statment_df['quarter'] = [pd.Timestamp(quarter) for quarter in income_statment_df['quarter']]
    income_statment_df['quarter'] = ['4Q2020' if quarter <= pd.Timestamp('2021-01-01T12') else
                                     '1Q2021' if quarter > pd.Timestamp('2021-01-02T12') and quarter < pd.Timestamp('2021-04-01T12') else
                                     '2Q2021' if quarter > pd.Timestamp('2021-04-02T12') and quarter < pd.Timestamp('2021-07-01T12') else
                                     '3Q2021' if quarter > pd.Timestamp('2021-07-02T12') and quarter < pd.Timestamp('2021-10-02T12') else
                                     '4Q2021' for quarter in income_statment_df['quarter']]

    income_statment_df.loc[:,'gross_profit_margin'] = income_statment_df['gross_profit_billion']/income_statment_df['revenue_billion']
    income_statment_df.loc[:,'net_profit_margin'] = income_statment_df['net_income_billion']/income_statment_df['revenue_billion']

    income_statment_df['ticker'] = ticker

    return income_statment_df.head(period)

In [447]:
def get_la_ratio(ticker, quarters):
  balance_sheet_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-balance-sheet"

  querystring = {"symbol":ticker,"region":"US"}

  headers = {
            'x-rapidapi-host': "yh-finance.p.rapidapi.com",
            'x-rapidapi-key': KEY
            }

  response = requests.request("GET", balance_sheet_url, headers=headers, params=querystring)

  balance_sheet_response = response.json()

  balance_df = pd.DataFrame()

  for item in balance_sheet_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    balance_line = pd.Series([item['endDate']['fmt'],
                              item['totalCurrentLiabilities']['raw'],
                              item['totalCurrentAssets']['raw']],
                              index = ['quarter','liabilities','assets'])

    balance_df = balance_df.append(balance_line, ignore_index = True)

  balance_df.loc[:,'la_ratio'] = round(balance_df['liabilities']/balance_df['assets'],2)
  balance_df.loc[:,'assets_billion'] = round(balance_df['assets']/1000000000,2)
  balance_df.loc[:,'liabilities_billion'] = round(balance_df['liabilities']/1000000000,2)
  balance_df = balance_df.drop(columns = ['assets','liabilities'])
  balance_df['ticker'] = ticker

  return balance_df.head(quarters)

In [448]:
def get_pe_ratio(ticker, quarters):
    stats_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-statistics"

    querystring = {"symbol":ticker,"region":"US"}

    headers = {
              'x-rapidapi-host': "yh-finance.p.rapidapi.com",
              'x-rapidapi-key': KEY
              }

    stats_response = requests.request("GET", stats_url, headers=headers, params=querystring).json()

    ratio_df = pd.DataFrame()

    if stats_response['timeSeries']['quarterlyPeRatio'] != []:
        for quarter in stats_response['timeSeries']['quarterlyPeRatio']:
            if quarter is not None:
                pd_row = pd.Series([quarter['asOfDate'], quarter['reportedValue']['fmt']],
                index = ['quarter','pe_ratio'])
                ratio_df = ratio_df.append(pd_row, ignore_index=True)

    if ratio_df.empty == True:
        ratio_df['ticker'] = [ticker]
        ratio_df['pe_ratio'] = [None]
        ratio_df['quarter'] = [datetime.now().date()]

    else:
        pass

    ratio_df['ticker'] = ticker

    return ratio_df.sort_values(by = 'quarter', ascending = False).head(quarters)

In [449]:
def list_perfomance(symbol):
    weekly_history_df = get_weekly_history_df(symbol)

    performance_list = [symbol,\
                        round((weekly_history_df.iloc[0]['close'] - weekly_history_df.iloc[-1]['close'])/weekly_history_df.loc[0]['close'],3),\
                        round(weekly_history_df['close'].std(),2),
                        round(weekly_history_df['close'].mean(),2),
                        round(weekly_history_df['close'].std()/weekly_history_df['close'].mean(),2),
                        round(weekly_history_df.iloc[0]['close'],1)
                        ]

    return performance_list

In [453]:
counter = 0

attributes = ['ticker','yoy_growth','std','mean_price','volatility','last_close_price']
comparison_df = pd.DataFrame(columns = attributes)

print('Geting year stock prices...')
for ticker_name in list(nasdaq100_df['ticker']):
    series_row = pd.Series(list_perfomance(ticker_name), index = attributes)
    comparison_df = comparison_df.append(series_row, ignore_index=True)
    print(ticker_name, end = ' ')

print('Done!')

la_ratio_main_df = pd.DataFrame()

print('Geting liabilities to assets ratio...')
for ticker_name in list(nasdaq100_df['ticker']):
    mini_df = get_la_ratio(ticker_name,1)
    la_ratio_main_df = la_ratio_main_df.append(mini_df)
    print(ticker_name, end = ' ')

print('Done!')

Geting year stock prices...
ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM Done!
Geting liabilities to assets ratio...
ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN

In [454]:
income_statement_main_df = pd.DataFrame()

print('Geting profitability ratio...')
for ticker_name in list(nasdaq100_df['ticker']):
    mini_df = get_income_statement(ticker_name,1)
    income_statement_main_df = income_statement_main_df.append(mini_df)
    print(ticker_name, end = ' ')

print('Done!')

pe_ratio_main_df = pd.DataFrame()

print('Geting price per earning ratio...')
for ticker_name in list(nasdaq100_df['ticker']):
    mini_df = get_pe_ratio(ticker_name,1)
    pe_ratio_main_df = pe_ratio_main_df.append(mini_df)
    print(ticker_name, end = ' ')
    time.sleep(1)

print('Done!')

Geting profitability ratio...
ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM Done!
Geting price per earning ratio...
ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN T

In [456]:
final_df = comparison_df.merge(la_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(income_statement_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(pe_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df.merge(nasdaq100_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df[['ticker','company_name','industry','sub_industry','last_close_price','yoy_growth','volatility',
                    'assets_billion','liabilities_billion','la_ratio','gross_profit_margin','net_profit_margin',
                    'pe_ratio']]

final_df

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
0,ATVI,Activision Blizzard,Communication Services,Communication Services,60.600,-0.289,0.110,11.210,2.020,0.180,0.745,0.309,23.24
1,ADBE,Adobe Inc.,Information Technology,Information Technology,662.100,0.280,0.140,8.620,6.190,0.720,0.881,0.308,57.56
2,AMD,Advanced Micro Devices,Information Technology,Information Technology,154.800,0.437,0.190,7.990,3.560,0.450,0.484,0.214,36.49
3,ALGN,Align Technology,Health Care,Health Care,634.200,0.250,0.110,2.460,1.790,0.730,0.743,0.178,76.22
4,GOOGL,Alphabet Inc.,Communication Services,Communication Services,2843.700,0.372,0.170,184.110,61.780,0.340,0.576,0.291,29.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,WBA,Walgreens Boots Alliance,Consumer Staples,Consumer Staples,45.900,0.150,0.080,15.810,22.050,1.390,0.219,0.018,19.59
98,WDAY,"Workday, Inc.",Information Technology,Information Technology,277.500,0.193,0.080,4.690,4.280,0.910,0.729,0.032,None
99,XEL,Xcel Energy,Utilities,Utilities,64.800,-0.043,0.050,4.490,5.760,1.280,0.453,0.176,21.26
100,XLNX,Xilinx,Information Technology,Information Technology,223.900,0.386,0.160,4.190,0.610,0.150,0.686,0.251,49.18


In [457]:
final_df.to_csv('/Users/ilya/Desktop/Computer-Science/Github/check-stock-portfolio/saved_df.csv')

In [ ]:
#considerations
#0. yoy_growth
#1. volatility
#2. la_ratio
#3. gross_profit_margin
#4. net_profit_margin
#5. pe_ratio

In [466]:
final_df = final_df.drop(columns = ['volatility_bin'])

In [492]:
analysis_df = final_df[['ticker','yoy_growth','volatility','la_ratio','gross_profit_margin','net_profit_margin']]

analysis_df

,ticker,yoy_growth,volatility,la_ratio,gross_profit_margin,net_profit_margin
0,ATVI,-0.289,0.110,0.180,0.745,0.309
1,ADBE,0.280,0.140,0.720,0.881,0.308
2,AMD,0.437,0.190,0.450,0.484,0.214
3,ALGN,0.250,0.110,0.730,0.743,0.178
4,GOOGL,0.372,0.170,0.340,0.576,0.291
...,...,...,...,...,...,...
97,WBA,0.150,0.080,1.390,0.219,0.018
98,WDAY,0.193,0.080,0.910,0.729,0.032
99,XEL,-0.043,0.050,1.280,0.453,0.176
100,XLNX,0.386,0.160,0.150,0.686,0.251


In [520]:
analysis_df.describe()

,yoy_growth,volatility,la_ratio,gross_profit_margin,net_profit_margin
count,101.000,102.000,102.000,102.000,102.000
mean,0.106,0.108,0.713,0.571,0.146
std,0.305,0.062,0.539,0.207,0.242
min,-1.349,0.040,0.140,-0.278,-0.877
25%,-0.004,0.060,0.340,0.463,0.106
50%,0.157,0.090,0.595,0.576,0.175
75%,0.262,0.130,0.865,0.729,0.279
max,0.615,0.440,3.500,0.917,0.671


In [494]:
la_ratio_top = analysis_df['la_ratio'].mean() + analysis_df['la_ratio'].std()
volatility_top = analysis_df['volatility'].mean() + analysis_df['volatility'].std()
yoy_growth_bottom = analysis_df['yoy_growth'].mean() - analysis_df['yoy_growth'].std()

In [521]:
la_analysis_df = analysis_df[analysis_df['la_ratio'] < la_ratio_top]
la_analysis_df.loc[:,'la_ratio_bin'] = pd.cut(la_analysis_df['la_ratio'], 8, precision = 1)
la_analysis_df2 = la_analysis_df[['ticker','la_ratio_bin']].groupby('la_ratio_bin').count().reset_index()
la_analysis_df2.loc[:,'la_ratio_bin'] = la_analysis_df2['la_ratio_bin'].astype('str')


fig = px.bar(la_analysis_df2, x='la_ratio_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#106e45',
                  textfont_size=14)

fig.update_layout(title_text = 'Liabilities to Assets Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'LA Ratio Bin',
                  height = 500, width = 1000)

fig.show()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [522]:
volatility_analysis_df = analysis_df[analysis_df['volatility'] < volatility_top]
volatility_analysis_df.loc[:,'volatility_bin'] = pd.cut(volatility_analysis_df['volatility'], 8, precision = 1)
volatility_analysis_df2 = volatility_analysis_df[['ticker','volatility_bin']].groupby('volatility_bin').count().reset_index()
volatility_analysis_df2.loc[:,'volatility_bin'] = volatility_analysis_df2['volatility_bin'].astype('str')

fig = px.bar(volatility_analysis_df2, x='volatility_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#9467bd',
                  textfont_size=14)

fig.update_layout(title_text = 'Volatility Ratio Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Volatility Bin',
                  height = 500, width = 1000)

fig.show()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [538]:
gpm_analysis_df = analysis_df[['ticker','gross_profit_margin']]
gpm_analysis_df.loc[:,'gross_profit_margin_bin'] = pd.cut(gpm_analysis_df['gross_profit_margin'], 8, precision = 1)
gpm_analysis_df2 = gpm_analysis_df[['gross_profit_margin_bin','ticker']].groupby('gross_profit_margin_bin').count().reset_index()
gpm_analysis_df2.loc[:,'gross_profit_margin_bin'] = gpm_analysis_df2['gross_profit_margin_bin'].astype('str')

fig = px.bar(gpm_analysis_df2, x='gross_profit_margin_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#000080',
                  textfont_size=14)

fig.update_layout(title_text = 'Gross Profit Margin Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Gross Profit Margin Bin',
                  height = 500, width = 1000)

fig.update_layout({'plot_bgcolor': 'rgba(192, 218, 218, 0)','paper_bgcolor': 'rgba(192, 218, 218, 0)',})

fig.show()

In [527]:
gpm_analysis_df = analysis_df[['ticker','gross_profit_margin']]
gpm_analysis_df.loc[:,'gross_profit_margin_bin'] = pd.cut(gpm_analysis_df['gross_profit_margin'], 8, precision = 1)
gpm_analysis_df2 = gpm_analysis_df[['gross_profit_margin_bin','ticker']].groupby('gross_profit_margin_bin').count()
gpm_analysis_df2.loc[:,'gross_profit_margin_bin'] = gpm_analysis_df2['gross_profit_margin_bin'].astype('str')

fig = px.bar(gpm_analysis_df2, x='gross_profit_margin_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#106e45',
                  textfont_size=14)

fig.update_layout(title_text = 'Gross Profit Margin Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Gross Profit Margin Bin',
                  height = 500, width = 1000)

fig.show()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError: 'gross_profit_margin_bin'

In [503]:
la_analysis_df.loc[:,'la_ratio_bin'] = pd.cut(la_analysis_df['gross_profit_margin'], 8, precision = 1)
la_analysis_df2 = la_analysis_df[['ticker','la_ratio_bin']].groupby('la_ratio_bin').count().reset_index()
la_analysis_df2.loc[:,'la_ratio_bin'] = la_analysis_df2['la_ratio_bin'].astype('str')

fig = px.bar(la_analysis_df2, x='la_ratio_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#106e45',
                  textfont_size=14)

fig.update_layout(title_text = 'Liabilities to Assets Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'LA Ratio Bin',
                  height = 500, width = 1000)

fig.show()

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
0,ATVI,Activision Blizzard,Communication Services,Communication Services,60.600,-0.289,0.110,11.210,2.020,0.180,0.745,0.309,23.24
2,AMD,Advanced Micro Devices,Information Technology,Information Technology,154.800,0.437,0.190,7.990,3.560,0.450,0.484,0.214,36.49
4,GOOGL,Alphabet Inc.,Communication Services,Communication Services,2843.700,0.372,0.170,184.110,61.780,0.340,0.576,0.291,29.01
5,GOOG,Alphabet Inc.,Communication Services,Communication Services,2856.100,0.372,0.170,184.110,61.780,0.340,0.576,0.291,28.92
9,ADI,Analog Devices,Information Technology,Information Technology,178.000,0.226,0.070,5.380,2.770,0.520,0.479,0.032,37.96
10,ANSS,Ansys,Information Technology,Information Technology,382.500,0.136,0.060,1.760,0.600,0.340,0.893,0.193,65.22
12,AMAT,Applied Materials,Information Technology,Information Technology,143.300,0.423,0.150,16.110,6.340,0.390,0.481,0.280,23.77
13,ASML,ASML Holding,Information Technology,Information Technology,782.000,0.440,0.180,15.680,9.160,0.580,0.517,0.332,55.54
17,BIDU,Baidu,Communication Services,Communication Services,153.100,0.089,0.220,215.390,73.920,0.340,0.495,-0.519,7.77
18,BIIB,Biogen,Health Care,Health Care,242.900,-0.004,0.140,7.160,4.210,0.590,0.816,0.118,21.99


In [481]:
analysis_df[['ticker','la_ratio_bin']].groupby('la_ratio_bin').count()

,ticker
la_ratio_bin,
"(0.1, 0.6]",54
"(0.6, 1.1]",35
"(1.1, 1.6]",7
"(1.6, 2.1]",3
"(2.1, 2.5]",1
"(2.5, 3.0]",1
"(3.0, 3.5]",1


In [452]:
final_df[(final_df['last_close_price'] <= 500) & (final_df['yoy_growth'] >= 0.05)]

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
0,ATVI,Activision Blizzard,Communication Services,Communication Services,60.600,-0.289,0.110,11.210,2.020,0.180,0.745,0.309,23.24
2,AMD,Advanced Micro Devices,Information Technology,Information Technology,154.800,0.437,0.190,7.990,3.560,0.450,0.484,0.214,36.49
7,AEP,American Electric Power,Utilities,Utilities,82.600,-0.028,0.040,5.790,9.950,1.720,0.469,0.172,17.31
8,AMGN,Amgen,Health Care,Health Care,201.100,-0.118,0.060,24.380,14.840,0.610,0.760,0.281,21.61
9,ADI,Analog Devices,Information Technology,Information Technology,178.000,0.226,0.070,5.380,2.770,0.520,0.479,0.032,37.96
10,ANSS,Ansys,Information Technology,Information Technology,382.500,0.136,0.060,1.760,0.600,0.340,0.893,0.193,65.22
11,AAPL,Apple Inc.,Information Technology,Information Technology,156.800,0.256,0.080,134.840,125.480,0.930,0.422,0.247,27.64
12,AMAT,Applied Materials,Information Technology,Information Technology,143.300,0.423,0.150,16.110,6.340,0.390,0.481,0.280,23.77
14,TEAM,Atlassian,Information Technology,Information Technology,391.500,0.428,0.270,2.010,2.400,1.190,0.840,-0.651,None
15,ADSK,Autodesk,Information Technology,Information Technology,254.200,-0.073,0.060,2.650,3.280,1.240,0.917,0.122,54.99
